# 0. Setup

In [1]:
%load_ext autoreload
%autoreload 2
import sys, json, copy
import logging
logging.disable(logging.CRITICAL)
sys.path.append("src/")

In [2]:
import os
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')
os.environ['AI21_STUDIO_API_KEY'] = 'AI21_STUDIO_API_KEY'
os.environ['COHERE_STUDIO_API_KEY'] = 'COHERE_STUDIO_API_KEY'
os.environ['TEXTSYNTH_API_SECRET_KEY'] = 'TEXTSYNTH_API_SECRET_KEY'
os.environ['ANTHROPIC_API_KEY'] = 'ANTHROPIC_API_KEY'
os.environ['TOGETHER_API_KEY'] = 'TOGETHER_API_KEY'

from IPython.display import display
import FrugalGPT
supported_LLM = FrugalGPT.getservicename()
print("supported LLMs:",supported_LLM)

/Users/sharkiefff/anaconda3/envs/dis-llm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/sharkiefff/anaconda3/envs/dis-llm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


supported LLMs: ['textsynth/gptneox_20B', 'textsynth/fairseq_gpt_13B', 'textsynth/gptj_6B', 'openai/text-davinci-002', 'openai/text-davinci-003', 'openai/text-curie-001', 'openai/text-babbage-001', 'openai/text-ada-001', 'openaichat/gpt-4o-mini', 'openaichat/gpt-4-turbo', 'openaichat/gpt-4o', 'openaichat/gpt-3.5-turbo', 'openaichat/gpt-4', 'ai21/j1-jumbo', 'ai21/j1-grande', 'ai21/j1-large', 'ai21/j2-ultra', 'ai21/j2-mid', 'ai21/j2-light', 'cohere/command', 'cohere/base', 'cohere/xlarge', 'cohere/medium', 'anthropic/claude-1', 'anthropic/claude-instant-1', 'anthropic/claude-1-100k']


In [ ]:
# check the key
print(os.environ['ANTHROPIC_API_KEY'])
print(os.environ['OPENAI_API_KEY'])

In [ ]:
# a test of openai api

from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            # "content": "Write a haiku about recursion in programming."
            "content": "What is the purpose of life?"
        }
    ]
)

print(completion.choices[0].message)

Notice: remember to modify the name in config/serviceinfo_thrift.json to adjust the llm model you want to use.

# 1. Prepare the Dataset

In [8]:
dataname = "HEADLINES"

test_data = FrugalGPT.loadcsvdata(f"data/{dataname}/test.csv")
prefix = open(f'config/prompt/{dataname}/prefix_e8.txt').read()
test_data = FrugalGPT.formatdata(test_data,prefix)

train_data = FrugalGPT.loadcsvdata(f"data/{dataname}/train.csv")
prefix = open(f'config/prompt/{dataname}/prefix_e8.txt').read()
train_data = FrugalGPT.formatdata(train_data,prefix)

# 2. Train a LLMCascade

Specify the budget per query, and then train the model. Warning: This can take a while on large datasets!

In [16]:
# train the model
MyCascade = FrugalGPT.LLMCascade()
service_names = ['ai21/j1-large'] 
# 'openaichat/gpt-4-turbo', 'openaichat/gpt-4o', 'openaichat/gpt-4o-mini'
# 'ai21/j1-large','textsynth/gptj_6B'

result = MyCascade.train(train_data,budget=100,service_names=service_names)

train and test size 4950 50


/Users/sharkiefff/anaconda3/envs/dis-llm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/sharkiefff/anaconda3/envs/dis-llm/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

  0%|          | 0/2232 [00:00<?, ?it/s]

{'loss': 0.733, 'grad_norm': 2.9268574714660645, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}
{'loss': 0.7205, 'grad_norm': 3.7032692432403564, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.05}
{'loss': 0.6916, 'grad_norm': 3.727867364883423, 'learning_rate': 3e-06, 'epoch': 0.08}
{'loss': 0.6589, 'grad_norm': 3.131098985671997, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.11}
{'loss': 0.6536, 'grad_norm': 2.136280059814453, 'learning_rate': 5e-06, 'epoch': 0.13}
{'loss': 0.5324, 'grad_norm': 3.764906883239746, 'learning_rate': 6e-06, 'epoch': 0.16}
{'loss': 0.5294, 'grad_norm': 2.3770830631256104, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.19}
{'loss': 0.5636, 'grad_norm': 2.98032283782959, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.22}
{'loss': 0.4954, 'grad_norm': 2.9582624435424805, 'learning_rate': 9e-06, 'epoch': 0.24}
{'loss': 0.553, 'grad_norm': 8.271008491516113, 'learning_rate': 1e-05, 'epoch': 0.27}
{'loss': 0.4161, 'grad_norm': 2.46609

  0%|          | 0/31 [00:00<?, ?it/s]

...

{'eval_loss': 0.467580646276474, 'eval_accuracy': 0.8878787878787879, 'eval_runtime': 2.6192, 'eval_samples_per_second': 755.943, 'eval_steps_per_second': 11.835, 'epoch': 4.0}
{'loss': 0.0351, 'grad_norm': 0.07322902232408524, 'learning_rate': 2.1420323325635107e-05, 'epoch': 4.01}
{'loss': 0.03, 'grad_norm': 36.649295806884766, 'learning_rate': 2.113163972286374e-05, 'epoch': 4.03}
{'loss': 0.0053, 'grad_norm': 0.07328525185585022, 'learning_rate': 2.084295612009238e-05, 'epoch': 4.06}
{'loss': 0.0658, 'grad_norm': 0.25959160923957825, 'learning_rate': 2.0554272517321016e-05, 'epoch': 4.09}
{'loss': 0.1338, 'grad_norm': 6.289260387420654, 'learning_rate': 2.0265588914549654e-05, 'epoch': 4.11}
{'loss': 0.1074, 'grad_norm': 49.256370544433594, 'learning_rate': 1.997690531177829e-05, 'epoch': 4.14}
{'loss': 0.0633, 'grad_norm': 0.07163897156715393, 'learning_rate': 1.968822170900693e-05, 'epoch': 4.17}
{'loss': 0.2159, 'grad_norm': 30.809186935424805, 'learning_rate': 1.939953810623556

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.5196784138679504, 'eval_accuracy': 0.8934343434343435, 'eval_runtime': 2.7869, 'eval_samples_per_second': 710.477, 'eval_steps_per_second': 11.124, 'epoch': 5.0}
{'loss': 0.0468, 'grad_norm': 0.057662397623062134, 'learning_rate': 1.0450346420323325e-05, 'epoch': 5.03}
{'loss': 0.0428, 'grad_norm': 0.031790196895599365, 'learning_rate': 1.0161662817551963e-05, 'epoch': 5.05}
{'loss': 0.1127, 'grad_norm': 14.367451667785645, 'learning_rate': 9.8729792147806e-06, 'epoch': 5.08}
{'loss': 0.1412, 'grad_norm': 13.23043441772461, 'learning_rate': 9.584295612009238e-06, 'epoch': 5.11}
{'loss': 0.0994, 'grad_norm': 0.033957589417696, 'learning_rate': 9.295612009237876e-06, 'epoch': 5.13}
{'loss': 0.0293, 'grad_norm': 0.027305850759148598, 'learning_rate': 9.006928406466514e-06, 'epoch': 5.16}
{'loss': 0.0021, 'grad_norm': 0.02560787834227085, 'learning_rate': 8.718244803695151e-06, 'epoch': 5.19}
{'loss': 0.0852, 'grad_norm': 0.036879103630781174, 'learning_rate': 8.42956120092

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.5322539210319519, 'eval_accuracy': 0.896969696969697, 'eval_runtime': 2.743, 'eval_samples_per_second': 721.847, 'eval_steps_per_second': 11.302, 'epoch': 6.0}
{'train_runtime': 221.4487, 'train_samples_per_second': 80.47, 'train_steps_per_second': 10.079, 'train_loss': 0.2328433177580366, 'epoch': 6.0}


In [17]:
# save to disk
MyCascade.save(savepath="strategy/EXPERIMENT/")

In [ ]:
MyCascade = FrugalGPT.LLMCascade()
MyCascade.load(loadpath="strategy/EXPERIMENT/",budget=100)

# 3. Performance

In [ ]:
print("test data size:",len(test_data))
MyCascade = FrugalGPT.LLMCascade()
MyCascade.load(loadpath="strategy/HEADLINES/",budget=0.000665)